## Grouby queries from h2o-benchmarks (parquet)

In [ ]:
import os

import coiled
import dask.dataframe as dd
from dask.distributed import Client, performance_report
import pandas as pd

In [ ]:
# Note: this cluster might be small for the 50GB 
cluster = coiled.Cluster(
    name="h2o-benchmarks",
    n_workers=10,
    worker_vm_types=["t3.large"],  # 2CPU, 8GiB
    scheduler_vm_types=["t3.large"], #maybe we should try t3.xlarge? , 
    package_sync=True,
)

In [ ]:
client = Client(cluster)
client

In [ ]:
data_size = {
    "05GB": "s3://coiled-datasets/h2o-benchmark/N_1e7_K_1e2_parquet/*.parquet",
    "5GB": "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_parquet/*.parquet",
    "50GB": "s3://coiled-datasets/h2o-benchmark/N_1e9_K_1e2_parquet/*.parquet",
    "05GB_id3NC": "s3://coiled-datasets/h2o-benchmark/id3_nocat/N_1e7_K_1e2_parquet/*.parquet",
    "5GB_id3NC": "s3://coiled-datasets/h2o-benchmark/id3_nocat/N_1e8_K_1e2_parquet/*.parquet",
    "50GB_id3NC": "s3://coiled-datasets/h2o-benchmark/id3_nocat/N_1e9_K_1e2_parquet/*.parquet",
}

In [ ]:
# id3NC stands for id3 No Categorical, id3 is string[python]
ds = "50GB_id3NC" # choose "05GB" , "5GB" or "50GB", 05GB_id3NC, 5GB_id3NC, 50GB_id3NC
report_dir = "performance-reports-no-cat"

In [ ]:
ddf = dd.read_parquet(
    data_size[ds],
    engine="pyarrow",
    storage_options={"anon": True},
)
ddf

### Memory usage and `dtype` inspection

Total size on disk and in memory

In [ ]:
from dask.utils import format_bytes
import s3fs

fs = s3fs.S3FileSystem()
disk_size = fs.du(os.path.dirname(data_size[ds]))
format_bytes(disk_size)

In [ ]:
in_memory_size = ddf.memory_usage(deep=True).sum().compute()
format_bytes(in_memory_size)

Data is ~4x larger in memory than on disk

In [ ]:
ddf.memory_usage_per_partition(deep=True).compute() / 1e6

In [ ]:
ddf.astype({"id3": "string[pyarrow]"}).memory_usage_per_partition(deep=True).compute() / 1e6

Column `id3` takes up the most memory (by far). This is a `categorical` with lots of categories. Using `string[pyarrow]` instead of `category` takes up ~2x less memory for the entire DataFrame.

In [ ]:
ddf.partitions[0].memory_usage(deep=True).compute() / 1e6

Converting to `string[pyarrow]` reduces memory of `id3` by 4x.

In [ ]:
ddf.partitions[0].astype({"id3": "string[pyarrow]"}).memory_usage(deep=True).compute() / 1e6

### General notes

Ian mentioned that the optimization for columns might not happen when we do things like 
`ddf.astype({"id3": "object"})[["id3", "v1", "v3"]]` . We might want to modify the notebook to explicitely pass the columns to the `read_parquet` call. 

### Q1

In [ ]:
with performance_report(filename=os.path.join(report_dir, f"q1_{ds}.html")):
    ddf_q1 = ddf[["id1", "v1"]]
    ddf_q1.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"}).compute()

### Q2

In [ ]:
with performance_report(filename=os.path.join(report_dir, f"q2_{ds}.html")):
    
    ddf_q2 = ddf[["id1", "id2", "v1"]]
    (
        ddf_q2.groupby(["id1", "id2"], dropna=False, observed=True)
        .agg({"v1": "sum"})
        .compute()
    )

### Q3 

"50GB" got

```python
KilledWorker: ("('aggregate-chunk-1352eeaf67172ec25f4661a39965a066-d11c92fa9cd76cfe076460d4406ad2c6', 1736)", <WorkerState 'tls://10.0.11.26:40507', name: h2o-benchmarks-worker-e7b4d2ced4, status: closed, memory: 0, processing: 45>)

WARNING:root:error sending AWS credentials to cluster: Timed out trying to connect to 
tls://10.0.10.223:38937 after 30 s
```

NOTES ("50GB"): 
- Using `shuffle="p2p"` in the `.agg(...)` call allows this query to finish with the "50 GB" dataset
- Changing `id3` to `string[python]` in the original data instead of categorical, still results in a KillWorker without `shuffle=p2p`
- `string[python]` or `string[pyarrow]` and `shuffle=p2p` don't work at the moment. 
- Having id3 saved as `string[python]` and reading it as object using astype, the query with p2p took a little bit less than 10 min (528.19 s)
- Using `shuffle="tasks"` and casting as objects finishes in ~12 min (12m 32s) but it takes a while, probably the cluster is not big enough. 
- Using `shuffle="tasks"` and NOT casting to object turns into `KilledWorker`. No idea why this is happening but it is weird that without casting `id3` as an object, results in worst performance than pure strings.

Some notes from looking at query 3 with the 5 GM dataset:

- Categoricals are less efficient than `string[python]` and `string[pyarrow]` types. At least when there are lots of categories. 

- Want to make it easier for the user to specify dtypes in `read_parquet`

- Can't use `shuffle="p2p"` with `string[pyarrow]`. Breaks for some reason. 

In [ ]:
# Original query 3
with performance_report(filename=os.path.join(report_dir, f"q3_{ds}.html")):
    
    ddf_q3 = ddf[["id3", "v1", "v3"]]
    (
        ddf_q3.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "sum", "v3": "mean"})
        .compute()
    )

In [ ]:
# Query 3 id3 is saved as string[python] try shuffle="tasks"
with performance_report(filename=os.path.join(report_dir, f"q3_{ds}-tasks-shuffle_id3nocat_obj.html")):
    

    ddf_q3 = ddf.astype({"id3": "object"})[["id3", "v1", "v3"]]
    (
        ddf_q3.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "sum", "v3": "mean"}, shuffle="tasks")
        .compute()
    )

In [ ]:
# Query 3 id3 is saved as string[python] but does not work with p2p, try astype object
with performance_report(filename=os.path.join(report_dir, f"q3_{ds}-p2p-shuffle_id3object.html")):
    

    ddf_q3 = ddf.astype({"id3": "object"})[["id3", "v1", "v3"]]
    (
        ddf_q3.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "sum", "v3": "mean"}, shuffle="p2p")
        .compute()
    )

In [ ]:
# Query 3 with `p2p` shuffling algorithm
with performance_report(filename=os.path.join(report_dir, f"q3_{ds}-p2p-shuffle.html")):
    
    ddf_q3 = ddf[["id3", "v1", "v3"]]
    (
        ddf_q3.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "sum", "v3": "mean"}, shuffle="p2p")
        .compute()
    )

In [ ]:
# Query 3 with `string[pyarrow]` dtype
with performance_report(filename=os.path.join(report_dir, f"q3_{ds}-pyarrow.html")):
    
    ddf_q3 = ddf.astype({"id3": "string[pyarrow]"})[["id3", "v1", "v3"]]
    (
        ddf_q3.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "sum", "v3": "mean"})
        .compute()
    )

In [ ]:
# Query 3 with `string[python]` dtype
with performance_report(filename=os.path.join(report_dir, f"q3_{ds}-python.html")):
    
    ddf_q3 = ddf.astype({"id3": "string[python]"})[["id3", "v1", "v3"]]
    (
        ddf_q3.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "sum", "v3": "mean"})
        .compute()
    )

### Q4

In [ ]:
with performance_report(filename=os.path.join(report_dir, f"q4_{ds}.html")):
    
    ddf_q4 = ddf[["id4", "v1", "v2", "v3"]]
    (
        ddf_q4.groupby("id4", dropna=False, observed=True)
        .agg({"v1": "mean", "v2": "mean", "v3": "mean"})
        .compute()
    )

### Q5
50GB we see some spilling about 5GB of spilling, some workers dying but restarting very close to finishing, untile we got a:

```python
KilledWorker: ("('aggregate-combine-0c4783a5d5d1ec1968c74fb8baec15e9', 0, 3)", <WorkerState 'tls://10.0.15.17:36179', name: h2o-benchmarks-worker-7d835dad59, status: closed, memory: 0, processing: 1>)
```

By looking at the data generations this is another case of high-cardinality but with integers. From the script,
```python
    data["id6"] = np.random.choice(int(num_rows / num_groups), size=size, replace=True)
```
- On 50GB and shuffle p2p the query running it as:
```python
ddf_q5 = ddf[["id6", "v1", "v2", "v3"]]
    (
        ddf_q5.groupby("id6", dropna=False, observed=True)
        .agg({"v1": "sum", "v2": "sum", "v3": "sum"}, shuffle="p2p")
        .compute()
    )
```

returns 

```
AttributeError: 'Index' object has no attribute 'head'
```
- On 50GB and shuffle p2p cating id6 to "int32" finishes in ~4 min (249.57 s)

- On 50GB and shuffle tasks finishes in ~6 min (364.40 s)


In [ ]:
with performance_report(filename=os.path.join(report_dir, f"q5_{ds}.html")):
    
    ddf_q5 = ddf[["id6", "v1", "v2", "v3"]]
    (
        ddf_q5.groupby("id6", dropna=False, observed=True)
        .agg({"v1": "sum", "v2": "sum", "v3": "sum"})
        .compute()
    )

In [ ]:
# AttributeError: 'Index' object has no attribute 'head' if reading data - known issue reported in dask #9476
# ddf_q5 = ddf[["id6", "v1", "v2", "v3"]]
with performance_report(filename=os.path.join(report_dir, f"q5_{ds}_shuffle_p2p.html")):
    
    ddf_q5 = ddf.astype({"id6": "int32"})[["id6", "v1", "v2", "v3"]]
    (
        ddf_q5.groupby("id6", dropna=False, observed=True)
        .agg({"v1": "sum", "v2": "sum", "v3": "sum"}, shuffle="p2p")
        .compute()
    )

In [ ]:
with performance_report(filename=os.path.join(report_dir, f"q5_{ds}_shuffle_tasks.html")):
    
    ddf_q5 = ddf[["id6", "v1", "v2", "v3"]]
    (
        ddf_q5.groupby("id6", dropna=False, observed=True)
        .agg({"v1": "sum", "v2": "sum", "v3": "sum"}, shuffle="tasks")
        .compute()
    )

### Q7 
50GB 
```python
KilledWorker: ("('aggregate-chunk-351c6302b956751c118f4e66ba112f29-25ffa2e10908bad5aab7bf2896ccaca1', 1742)", <WorkerState 'tls://10.0.10.247:40875', name: h2o-benchmarks-worker-fa206d90a7, status: closed, memory: 0, processing: 40>)
```
Notes:
- Having id3 saved as `string[python]` and reading it as object using astype, the query with p2p took 472.83 s (~8 min)

In [ ]:
with performance_report(filename=os.path.join(report_dir, f"q7_{ds}.html")):
    
    ddf_q7 = ddf[["id3", "v1", "v2"]]
    (
        ddf_q7.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "max", "v2": "min"})
        .assign(range_v1_v2=lambda x: x["v1"] - x["v2"])[["range_v1_v2"]]
        .compute()
    )

In [ ]:
# Q7 with p2p and id3 astype object
with performance_report(filename=os.path.join(report_dir, f"q7_{ds}-p2p-shuffle_id3object.html")):
    
    ddf_q7 = ddf.astype({"id3": "object"})[["id3", "v1", "v2"]]
    (
        ddf_q7.groupby("id3", dropna=False, observed=True)
        .agg({"v1": "max", "v2": "min"}, shuffle="p2p")
        .assign(range_v1_v2=lambda x: x["v1"] - x["v2"])[["range_v1_v2"]]
        .compute()
    )

### Q8
Takes a long time in general. 

In [ ]:
with performance_report(filename=os.path.join(report_dir, f"q8_{ds}.html")):
    
    ddf_q8 = ddf[["id6", "v1", "v2", "v3"]]
    (
        ddf_q8[~ddf_q8["v3"].isna()][["id6", "v3"]]
        .groupby("id6", dropna=False, observed=True)
        .apply(
            lambda x: x.nlargest(2, columns="v3"),
            meta={"id6": "Int64", "v3": "float64"},
        )[["v3"]]
        .compute()
    )

### Q9

In [ ]:
with performance_report(filename=os.path.join(report_dir, f"q9_{ds}.html")):
    
    ddf_q9 = ddf[["id2", "id4", "v1", "v2"]]
    (
        ddf_q9[["id2", "id4", "v1", "v2"]]
        .groupby(["id2", "id4"], dropna=False, observed=True)
        .apply(
            lambda x: pd.Series({"r2": x.corr()["v1"]["v2"] ** 2}),
            meta={"r2": "float64"},
        )
        .compute()
    )

In [ ]:
cluster.close()